In [10]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-V6'
device = 'cuda'

In [30]:
data = pd.read_csv('./data.csv')[:2500].sample(frac=1)

In [31]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [32]:
tokenize("$100")

['$', '100']

In [33]:
def stem(word):
    return stemmer.stem(word.lower())

In [34]:
stem('organic')

'organ'

In [35]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [36]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [37]:
X = data['blurb']
y = data['state']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [38]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [39]:
labels

{'failed': 1, 'successful': 2}

In [40]:
labels_r

{1: 'failed', 2: 'successful'}

In [41]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

2500it [00:00, 2644.54it/s]


In [42]:
words = sorted(set(words))
np.random.shuffle(data)

In [43]:
X = []
y = []

In [44]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

100%|██████████| 2500/2500 [00:04<00:00, 556.49it/s]


In [45]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [46]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [47]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [48]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 64
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.linear2(preds)
        preds = self.linear3(preds)
        return preds

In [49]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [50]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


100%|██████████| 100/100 [00:18<00:00,  5.43it/s]


Acc,▁███████████████████████████████████████
Loss,█▂▅▂▆▂▆▂▂▁▁▁▁▁▁▁▁▁▂▄▂▂▂▂▁▁▁▁▁▁▂▁▂▂▁▁▂▂▂▁
Val Acc,▄█▅▆▅▅▁▇▅▄▄▃▆▆▆▇▇▃▆▆▆▃▇▇▄▄▅▄▄▅▄▇▃▂▅▅▁▃▄▅
Val Loss,▁▂▁▂█▃▆▂▅▃▄▃▃▃▂▃▃▃▂▇▅▃▄▄▄▄▄▄▄▄▄▃▄▄▃▃▅▃▃▃
Acc,100.0
Loss,0.0105
Val Acc,87.5
Val Loss,0.10944


In [51]:
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')